In [1]:
import torch
import torch.nn as nn
import numpy as np
import seaborn as sns

In [3]:
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [6]:
data = torch.tensor(iris[iris.columns[0:4]].values).float()

labels = torch.zeros(len(data), dtype=torch.long)

labels[iris.species=='versicolor'] = 1
labels[iris.species=='virginica'] = 2

In [14]:
proptraining = .8
nTraining = int(len(labels)*proptraining)

traintestBool = np.zeros(len(labels), dtype=bool)

items2user4train = np.random.choice(range(len(labels)), nTraining, replace=False)

traintestBool[items2user4train] = True

traintestBool

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False, False,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False, False,  True,
        True,  True, False,  True, False,  True,  True, False, False,
        True, False, False,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True, False,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True, False, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True, False,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,

In [15]:
print(torch.mean(labels.float()))


print(torch.mean(labels[traintestBool].float()))

print(torch.mean(labels[~traintestBool].float()))

tensor(1.)
tensor(1.0250)
tensor(0.9000)


In [16]:
ANNiris = nn.Sequential(
    nn.Linear(4, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 3),   
)

lossfun = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(params=ANNiris.parameters(), lr=0.01)

In [17]:
print(data.shape)

print(data[traintestBool, :].shape)

print(data[~traintestBool].shape)

torch.Size([150, 4])
torch.Size([120, 4])
torch.Size([30, 4])


In [18]:
numEpochs = 1000

losses = torch.zeros(numEpochs)
ongoingAcc = []

for epochi in range(numEpochs):
    yHat = ANNiris(data[traintestBool, :])

    ongoingAcc.append(100 * torch.mean((torch.argmax(yHat, axis=1) == labels[traintestBool]).float()))

    loss = lossfun(yHat, labels[traintestBool])
    losses[epochi] = loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [21]:
predictions = ANNiris(data[traintestBool, :])
trainacc = 100*torch.mean((torch.argmax(predictions, axis=1) == labels[traintestBool]).float())

predictions = ANNiris(data[~traintestBool, :])
testacc = 100 * torch.mean((torch.argmax(predictions,axis=1) == labels[~traintestBool]).float())

In [23]:
print("Final train accuracy: %g%%" %trainacc)
print("Final test accuracy: %g%%" %testacc)

Final train accuracy: 97.5%
Final test accuracy: 96.6667%
